In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline
from keras.applications.resnet50 import ResNet50

from keras.preprocessing import image                  
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.resnet50 import preprocess_input, decode_predictions

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True    

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,ZeroPadding2D,Activation
from keras.layers import Dropout, Flatten, Dense,BatchNormalization
from keras.models import Sequential,Model,model_from_json
from keras import optimizers
from keras.callbacks import ModelCheckpoint  
from keras import layers
from keras import regularizers


Using TensorFlow backend.


In [3]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets
train_files, train_targets = load_dataset('/Users/olivia/Desktop/dogapp/dogImages/train')
valid_files, valid_targets = load_dataset('/Users/olivia/Desktop/dogapp/dogImages/valid')
test_files, test_targets = load_dataset('/Users/olivia/Desktop/dogapp/dogImages/test')
dog_names = [item[27:-1] for item in sorted(glob("/Users/olivia/Desktop/dogapp/dogImages/train/*/"))]
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [39]:
# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

In [5]:
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

In [6]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151))

In [7]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [8]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:06<00:00, 125.33it/s]


In [9]:
reg=10**(np.random.uniform(-4,0))
model = Sequential()
### TODO: Define your architecture.
model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu',input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=3))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3))

model.add(Conv2D(128, (3, 3)))
#Batch Normalization normalizes each batch by both mean and variance reference.
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=3))


model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())#Flatten

model.add(Dense(512,activation='relu',kernel_regularizer=regularizers.l2(reg)))
model.add(Dropout(0.4))
model.add(Dense(len(dog_names), activation='softmax'))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 74, 74, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 6, 128)        

In [14]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
epochs = 15

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='/Users/olivia/Desktop/dogapp/saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/15
 800/6680 [==>...........................] - ETA: 1:02 - loss: 3.1501 - accuracy: 0.2025

KeyboardInterrupt: 

In [21]:
model.load_weights('/Users/olivia/Desktop/dogapp/saved_models/weights.best.from_scratch.hdf5')

In [22]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 16.7464%


In [24]:
bottleneck_features = np.load('/Users/olivia/Desktop/dogapp/bottleneck_features/DogVGG16Data.npz')
train_VGG16 = bottleneck_features['train']
valid_VGG16 = bottleneck_features['valid']
test_VGG16 = bottleneck_features['test']

In [25]:
VGG16_model = Sequential()
VGG16_model.add(GlobalAveragePooling2D(input_shape=train_VGG16.shape[1:]))
VGG16_model.add(Dense(133, activation='softmax'))
VGG16_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 133)               68229     
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


In [26]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [26]:
checkpointer = ModelCheckpoint(filepath='/Users/olivia/Desktop/dogapp/saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(train_VGG16, train_targets, 
          validation_data=(valid_VGG16, valid_targets),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/20
6680/6680 [==============================] - 1s 102us/step - loss: 2.1732 - accuracy: 0.5969 - val_loss: 2.4809 - val_accuracy: 0.5689

Epoch 00001: val_loss improved from inf to 2.48094, saving model to /Users/zimu/Desktop/saved_models/weights.best.VGG16.hdf5
Epoch 2/20
6680/6680 [==============================] - 1s 124us/step - loss: 1.2440 - accuracy: 0.7334 - val_loss: 2.0761 - val_accuracy: 0.6491

Epoch 00002: val_loss improved from 2.48094 to 2.07605, saving model to /Users/zimu/Desktop/saved_models/weights.best.VGG16.hdf5
Epoch 3/20
6680/6680 [==============================] - 1s 112us/step - loss: 0.8000 - accuracy: 0.8175 - val_loss: 1.8658 - val_accuracy: 0.6731

Epoch 00003: val_loss improved from 2.07605 to 1.86578, saving model to /Users/zimu/Desktop/saved_models/weights.best.VGG16.hdf5
Epoch 4/20
6680/6680 [==============================] - 1s 113us/step - loss: 0.5569 - accuracy: 0.8603 - val_loss: 1.8878 - val_

In [35]:
VGG16_model.load_weights('/Users/olivia/Desktop/dogapp/saved_models/weights.best.VGG16.hdf5')

In [40]:
VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 72.8469%


In [41]:
def VGG16_predict_breed(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = VGG16_model.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [42]:
#save model to be imported in the app

VGG16_model.save("/Users/olivia/Desktop/dogapp/saved_models/VGG16_model.h5")


In [43]:
from keras.models import load_model
# returns a compiled model identical to the previous one
VGG16_model= load_model('/Users/olivia/Desktop/dogapp/saved_models/VGG16_model.h5')

In [32]:
def draw_img(img_path):
    # load color (BGR) image
    img = cv2.imread(img_path)
   
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # display the image
    plt.imshow(cv_rgb)
    plt.show()

In [33]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.

def detect_dog(img_path):
    """determines whether the image contains a dog, or nothing and print results

    Parameters:
    img_path (str): path to image file

    """
    isDog = dog_detector(img_path)
    if isDog:
        draw_img(img_path)
        print('Dog detected!, the predecited dog breed is : ',VGG16_predict_breed(img_path))
        return 'Dog'
        
    else:
            print('Error! Could not detect a face or a dog in the provided image')
            return None

        